In [1]:
import boto3 
import sagemaker
from ibov.utils import load_config
from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel

### Loading Configs

In [2]:
# Load config dict
config = load_config()

In [3]:
role = config.get("sagemaker").get("role")
prefix = config.get("sagemaker").get("bucket_prefx")
data_dir = config.get("data").get("dir")
region = config.get("sagemaker").get("region")

In [89]:
dropout = config.get("model").get("dropout")
window = config.get("feature").get("window")
hidden_layer = config.get("model").get("hidden_layer")
lr = config.get("model").get("lr")
seed = config.get("model").get("seed")
epochs = config.get("model").get("epochs")

### Upload Data

In [4]:
session = sagemaker.Session(boto_session=boto3.session.Session(region_name=region))
bucket = session.default_bucket()

In [5]:
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
input_config = session.upload_data(path="config.json", bucket=bucket, key_prefix=prefix)

### Train Model

In [94]:
estimator = PyTorch(entry_point="train.py", 
                    source_dir="ibov", 
                    py_version="py3",
                    role=role, 
                    framework_version='0.4.0',
                    instance_count=1, 
                    instance_type='ml.p2.xlarge',
                    hyperparameters = {
                        "batch-size": 50,
                        "epochs": epochs,
                        "seed": seed,
                        "input-layer": window,
                        "hidden-layer": hidden_layer,
                        "dropout": dropout                   
                        
                        
                    })

In [95]:
estimator.fit({'train': input_data, "config": input_config})

2021-01-16 19:20:44 Starting - Starting the training job...
2021-01-16 19:21:07 Starting - Launching requested ML instancesProfilerReport-1610824841: InProgress
......
2021-01-16 19:22:17 Starting - Preparing the instances for training.........
2021-01-16 19:23:56 Downloading - Downloading input data...
2021-01-16 19:24:40 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-01-16 19:24:41,441 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-01-16 19:24:41,466 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-01-16 19:24:42,910 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-01-16 19:24:43,267 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-01-16 19:24:43,267 sagemaker-containers INFO     G

19:25:06, epoch: 6, train: 0.685, valid: 0.076
19:25:08, epoch: 7, train: 0.493, valid: 0.04
19:25:11, epoch: 8, train: 0.499, valid: 0.038
19:25:14, epoch: 9, train: 0.476, valid: 0.041
19:25:18, epoch: 10, train: 0.424, valid: 0.045
19:25:21, epoch: 11, train: 0.345, valid: 0.033
19:25:25, epoch: 12, train: 0.309, valid: 0.028
19:25:30, epoch: 13, train: 0.286, valid: 0.033
19:25:35, epoch: 14, train: 0.215, valid: 0.022
19:25:40, epoch: 15, train: 0.232, valid: 0.045
19:25:45, epoch: 16, train: 0.321, valid: 0.053
19:25:51, epoch: 17, train: 0.313, valid: 0.102
19:25:57, epoch: 18, train: 0.312, valid: 0.025
19:26:04, epoch: 19, train: 0.28, valid: 0.032
19:26:11, epoch: 20, train: 0.262, valid: 0.074
19:26:18, epoch: 21, train: 0.257, valid: 0.024
19:26:25, epoch: 22, train: 0.243, valid: 0.028
19:26:33, epoch: 23, train: 0.237, valid: 0.06
19:26:41, epoch: 24, train: 0.218, valid: 0.022
2021-01-16 19:26:41,616 sagemaker-containers INFO     Reporting training SUCCESS

2021-01-16 19

In [105]:
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     py_version="py3", 
                     entry_point='server.py',
                     source_dir='ibov')

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------

In [96]:
estimator.model_data

's3://sagemaker-us-east-1-977053370764/sagemaker-pytorch-2021-01-16-19-20-39-951/output/model.tar.gz'

In [13]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.large')

KeyboardInterrupt: 

In [59]:
import os
import io
import boto3
import numpy as np
import pandas as pd

In [60]:
client = boto3.client("sagemaker-runtime")

In [61]:
ibovespa = pd.read_csv(os.path.join(data_dir, "data.csv"))

In [79]:
asd =bytearray(np.array(ibovespa.iloc[0]))

In [64]:
payload = io.StringIO()
ibovespa.to_csv(payload, index=None)

In [85]:
with open(os.path.join(data_dir, "data.csv")) as f:
    saddd = f.read().encode('utf-8')

In [87]:
client.invoke_endpoint(EndpointName='sagemaker-pytorch-2021-01-16-18-20-47-583',
                       Body=saddd, #payload.getvalue(),
                       #ContentType='text/csv',
                       Accept='Accept')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<html>
  <head>
    <title>Internal Server Error</title>
  </head>
  <body>
    <h1><p>Internal Server Error</p></h1>
    
  </body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-pytorch-2021-01-16-18-20-47-583 in account 977053370764 for more information.

In [9]:
model = PyTorchModel(model_data=model_data, 
                     role=role, 
                     entry_point="ibov/train.py", 
                     py_version="py3",
                     framework_version='0.4.0')

NameError: name 'model_data' is not defined

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.large')

predictor = estimator.deploy(initial_instance_count=1,instance_type="ml.m4.xlarge")